<a href="https://colab.research.google.com/github/nlquantumm-source/RT-Control-Readout-Electronics/blob/main/RT_Control_%26_Readout_Electronics_MuMax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ## 1. Install and Compile MuMax3 (Custom CUDA 12 Build for L4 GPU)
#
# The NVIDIA L4 GPU (Compute Capability 8.9) requires modern CUDA 12 libraries.
# We compile from source, patch for CUDA 12, and explicitly generate kernels for the L4.

import os
import subprocess
import sys
import shutil

# Set working directory
os.chdir("/content")

# 1. Install Build Tools & Update Go
print("Installing build tools and updating Go...")
subprocess.run("apt update -qq && apt install -qq git build-essential wget", shell=True, check=True)

# Download and install Go 1.23 manually
if not os.path.exists("/usr/local/go/bin/go"):
    subprocess.run("wget -q https://go.dev/dl/go1.23.0.linux-amd64.tar.gz", shell=True, check=True)
    subprocess.run("tar -C /usr/local -xzf go1.23.0.linux-amd64.tar.gz", shell=True, check=True)
    subprocess.run("rm go1.23.0.linux-amd64.tar.gz", shell=True, check=True)

# FORCE system-wide Go version update by symlinking
# This ensures 'make' calls the correct 'go' binary
if os.path.exists("/usr/bin/go"):
    subprocess.run("rm /usr/bin/go", shell=True)
subprocess.run("ln -s /usr/local/go/bin/go /usr/bin/go", shell=True)

# Check for NVCC
try:
    subprocess.run(["nvcc", "--version"], check=True, capture_output=True)
    print("NVCC found.")
except:
    print("WARNING: NVCC not found. GPU kernels cannot be built.")

# 2. Clean and Clone MuMax3 Source Code
print("Cloning MuMax3 source...")
if os.path.exists("3"):
    shutil.rmtree("3")
subprocess.run("git clone https://github.com/mumax/3", shell=True, check=True)

# 3. PATCH: Fix compatibility with CUDA 12
print("Patching source code for CUDA 12 compatibility...")
dummy_mode_go = """
package cufft
type CompatibilityMode int
const (
	CompatNative CompatibilityMode = 0
	CompatFFTWPadding CompatibilityMode = 1
	CompatFFTWAsymmetric CompatibilityMode = 2
	CompatFFTWAll CompatibilityMode = 3
)
"""
with open("3/cuda/cufft/mode.go", "w") as f:
    f.write(dummy_mode_go)

# 4. PATCH: Update CUDA Targets for L4 (sm_89)
print("Updating CUDA targets in Makefile...")
cuda_dir = os.path.abspath("3/cuda")
makefile_path = os.path.join(cuda_dir, "Makefile")

if os.path.exists(makefile_path):
    with open(makefile_path, "r") as f:
        content = f.read()

    # Ensure sm_89 is present in targets.
    # We replace a common target like sm_75 to inject ours.
    target_flag = "-gencode arch=compute_89,code=sm_89 -gencode arch=compute_80,code=sm_80"

    if "-gencode" in content:
        if "sm_89" not in content:
            for arch in ["sm_75", "sm_70", "sm_60"]:
                if arch in content:
                    print(f"Injecting L4 targets alongside {arch}...")
                    content = content.replace(arch, f"{arch} {target_flag}")
                    break
    else:
        # Fallback if structure is different
        content = content.replace("nvcc", f"nvcc {target_flag}")

    with open(makefile_path, "w") as f:
        f.write(content)

    # Force run make using the new Go environment
    print("Running make in 3/cuda (generating kernels)...")
    try:
        # We explicitly pass the PATH just in case, though symlink handles it
        new_env = os.environ.copy()
        new_env["PATH"] = "/usr/local/go/bin:" + new_env["PATH"]
        subprocess.run("make", cwd=cuda_dir, shell=True, check=True, env=new_env)
        print("Kernels generated successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Make failed with error {e.returncode}.")

else:
    print("CRITICAL: No Makefile found in 3/cuda.")

# 5. Compile and Install Main Binary
print("Compiling MuMax3 Binary... (This takes 1-2 minutes)")
# Install the binary using the correct environment
subprocess.run("go install -v ./cmd/mumax3", cwd="/content/3", shell=True, check=True)

# 6. Move binary to path
subprocess.run("cp /root/go/bin/mumax3 /usr/local/bin/mumax3", shell=True)
subprocess.run("chmod +x /usr/local/bin/mumax3", shell=True)

# 7. Verify Installation
print("Verifying installation...")
subprocess.run("mumax3 -v", shell=True)

# 8. Python dependencies
subprocess.run("pip install -q numpy matplotlib ipywidgets ubermag", shell=True)

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

Installing build tools and updating Go...
NVCC found.
Cloning MuMax3 source...
Patching source code for CUDA 12 compatibility...
Updating CUDA targets in Makefile...
Running make in 3/cuda (generating kernels)...
Make failed with error 2.
Compiling MuMax3 Binary... (This takes 1-2 minutes)
Verifying installation...


In [ ]:
# ## 2. Define Equations with Interactive Inputs
#
# Use sliders for parameters in the equations.

# Parameters for 2.3: QPSK Modulation
omega_LO_slider = widgets.FloatSlider(value=2*np.pi*5e9, min=1e9, max=10e9, step=1e8, description='ω_LO (rad/s)')
I_amp_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, description='I(t) Amp')
Q_amp_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, description='Q(t) Amp')
t_max_slider = widgets.FloatSlider(value=1e-9, min=1e-10, max=10e-9, step=1e-10, description='t_max (s)')
dt_slider = widgets.FloatSlider(value=1e-12, min=1e-13, max=1e-11, description='dt (s)')

# Parameters for 2.4: Nonlinear Coefficients
alpha1_slider = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, description='α₁')
alpha2_slider = widgets.FloatSlider(value=0.1, min=0, max=0.5, description='α₂')
alpha3_slider = widgets.FloatSlider(value=0.01, min=0, max=0.1, description='α₃')

# Display widgets
display(omega_LO_slider, I_amp_slider, Q_amp_slider, t_max_slider, dt_slider)
display(alpha1_slider, alpha2_slider, alpha3_slider)

# Function to generate x(t) from 2.3 (simple constant I/Q for demo; can extend to waveforms)
def generate_xt(omega_LO, I_amp, Q_amp, t_max, dt):
    t = np.arange(0, t_max, dt)
    I_t = np.full_like(t, I_amp)  # Constant for simplicity; replace with s(t) cos(phi0), etc.
    Q_t = np.full_like(t, Q_amp)
    x_t = I_t * np.cos(omega_LO * t) - Q_t * np.sin(omega_LO * t)
    return t, x_t

# Function for y(t) from 2.4 nonlinearity
def apply_nonlinearity(x_t, alpha1, alpha2, alpha3):
    y_t = alpha1 * x_t + alpha2 * x_t**2 + alpha3 * x_t**3
    return y_t

# Compute 1 dB compression and IP3
def compute_metrics(alpha1, alpha3):
    A_1dB = np.sqrt(0.145 * np.abs(alpha1 / alpha3)) if alpha3 != 0 else np.inf
    A_IP3 = np.sqrt((4/3) * np.abs(alpha1 / alpha3)) if alpha3 != 0 else np.inf
    return A_1dB, A_IP3

# Interactive plot of signals
@interact
def plot_signals(omega_LO=omega_LO_slider, I_amp=I_amp_slider, Q_amp=Q_amp_slider,
                 t_max=t_max_slider, dt=dt_slider,
                 alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider):
    t, x_t = generate_xt(omega_LO, I_amp, Q_amp, t_max, dt)
    y_t = apply_nonlinearity(x_t, alpha1, alpha2, alpha3)
    A_1dB, A_IP3 = compute_metrics(alpha1, alpha3)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    ax1.plot(t * 1e9, x_t, label='x(t) [Ideal QPSK]')
    ax1.set_xlabel('Time (ns)')
    ax1.set_ylabel('Amplitude')
    ax1.legend()
    ax1.grid(True)

    ax2.plot(t * 1e9, y_t, label='y(t) [Nonlinear]', color='r')
    ax2.set_xlabel('Time (ns)')
    ax2.set_ylabel('Amplitude')
    ax2.legend()
    ax2.grid(True)

    plt.suptitle(f'RF Signals: 1dB Point={A_1dB:.2f}, IP3={A_IP3:.2f}')
    plt.tight_layout()
    plt.show()

FloatSlider(value=10000000000.0, description='ω_LO (rad/s)', max=10000000000.0, min=1000000000.0, step=1000000…

FloatSlider(value=1.0, description='I(t) Amp', max=5.0, min=0.1)

FloatSlider(value=1.0, description='Q(t) Amp', max=5.0, min=0.1)

FloatSlider(value=1e-09, description='t_max (s)', max=1e-08, min=1e-10, step=1e-10)

FloatSlider(value=1e-12, description='dt (s)', max=1e-11, min=1e-13)

FloatSlider(value=1.0, description='α₁', max=2.0, min=0.5)

FloatSlider(value=0.1, description='α₂', max=0.5)

FloatSlider(value=0.01, description='α₃', max=0.1)

interactive(children=(FloatSlider(value=10000000000.0, description='ω_LO (rad/s)', max=10000000000.0, min=1000…

In [ ]:
# ## 3. Prepare MuMax3 Input
#
# Create a .mx3 script for a ferromagnetic cube driven by Hx(t) = y(t).
# We use .mx3 extension to ensure the interpreter handles it, avoiding Go compilation errors.

# MuMax3 input script template
# Updated to use // for comments (Go syntax) instead of #
# Updated TimeStep -> MaxDt
mumax_template = """
// Geometry and material
SetGridSize(50, 50, 5)
SetCellSize(2e-9, 2e-9, 2e-9)

// Permalloy-like parameters
Msat = 800e3
Alpha = 0.01
Aex = 13e-12

// Initial state (Uniform magnetization along Z)
m = Uniform(0, 0, 1)

// Define time-dependent drive parameters
// We inject calculated constants directly
w_LO  := {omega}
I_amp := {I}
Q_amp := {Q}
a1    := {a1}
a2    := {a2}
a3    := {a3}
scale := {scale}  // Conversion to Tesla

// Define signals as functions of time (t)
// x(t) = I*cos(w*t) - Q*sin(w*t)
xt := I_amp * cos(w_LO * t) - Q_amp * sin(w_LO * t)

// y(t) = a1*x + a2*x^2 + a3*x^3
yt := a1 * xt + a2 * xt*xt + a3 * xt*xt*xt

// Apply as external magnetic field B_ext (Tesla)
// Drive along X-axis
B_ext = vector(yt * scale, 0, 0)

// Time stepping configuration
t = 0
MaxDt = {dt}  // Limit maximum time step for RF resolution
AutoSave(m, {save_period})  // Save magnetization at fixed interval

// Run simulation
Run({t_max})
Save(m)  // Ensure final state is saved
"""

# Function to write the script with current slider values
def write_mumax_script(t_max, dt):
    # 1. Get values from interactive sliders
    omega = omega_LO_slider.value
    I_val = I_amp_slider.value
    Q_val = Q_amp_slider.value
    a1 = alpha1_slider.value
    a2 = alpha2_slider.value
    a3 = alpha3_slider.value

    # 2. Scale factor: slider amp -> Tesla
    H_scale = 0.001

    # 3. Pre-calculate save period to avoid script math issues
    # Save 20 frames total or at least every 20 steps
    save_period = dt * 20

    # 4. Format the script
    script = mumax_template.format(
        omega=omega, I=I_val, Q=Q_val,
        a1=a1, a2=a2, a3=a3, scale=H_scale,
        t_max=t_max, dt=dt, save_period=save_period
    )

    # 5. Write to file (Use .mx3 extension)
    with open('simulation.mx3', 'w') as f:
        f.write(script)
    print("Generated 'simulation.mx3' with analytical drive.")

In [ ]:
# ## 4. Run MuMax3 Simulation
#
# Run the simulation automatically.

def run_mumax(t_max, dt):
    # 1. Generate the script
    write_mumax_script(t_max, dt)

    # Verify the script content (Debugging check)
    with open('simulation.mx3', 'r') as f:
        content = f.read()
        if "MaxDt" in content:
            print("Script syntax check: 'MaxDt' parameter found (Correct).")
        else:
            print("Warning: 'MaxDt' missing. Please re-run Step 3.")

    print("Checking MuMax3 health... ")
    try:
        # Sanity check
        sanity_script = "SetGridSize(4,4,1); SetCellSize(1e-9,1e-9,1e-9); m=Uniform(1,0,0); Run(1e-15)"
        subprocess.run(['mumax3', '-s'], input=sanity_script, check=True, text=True, capture_output=True)
        print("Health check passed. Starting main simulation...")
    except subprocess.CalledProcessError as e:
        print("\nCRITICAL ERROR: MuMax3 failed to run on this GPU.")
        print("Diagnostics (STDERR):\n", e.stderr)
        return

    # 2. Run main simulation
    print("Running Simulation... (Please wait)")
    try:
        # Remove old output to avoid confusion
        if os.path.exists('simulation.out'):
            import shutil
            shutil.rmtree('simulation.out')

        result = subprocess.run(
            ['mumax3', 'simulation.mx3'],
            check=True,
            capture_output=True,
            text=True
        )
        print("Simulation completed successfully!")
        print("Output data saved to 'simulation.out'. Run Step 5 to visualize.")
    except subprocess.CalledProcessError as e:
        print(f"\nError: Simulation failed with return code {e.returncode}.")
        print("--- STDERR ---")
        print(e.stderr)
        print("--- STDOUT ---")
        print(e.stdout)

# Interactive run button
run_button = widgets.Button(description="Run MuMax3", button_style='success')
def on_run_clicked(b):
    run_button.disabled = True
    try:
        run_mumax(t_max_slider.value, dt_slider.value)
    finally:
        run_button.disabled = False

run_button.on_click(on_run_clicked)
display(run_button)

# AUTO-RUN: Trigger immediately for user convenience
print("Auto-starting simulation with fixed script...")
run_mumax(t_max_slider.value, dt_slider.value)

Button(button_style='success', description='Run MuMax3', style=ButtonStyle())

Auto-starting simulation with fixed script...
Generated 'simulation.mx3' with analytical drive.
Script syntax check: 'MaxDt' parameter found (Correct).
Checking MuMax3 health... 

CRITICAL ERROR: MuMax3 failed to run on this GPU.
Diagnostics (STDERR):
 
No binary for GPU. Your nvidia driver may be out-of-date


Generated 'simulation.mx3' with analytical drive.
Script syntax check: 'MaxDt' parameter found (Correct).
Checking MuMax3 health... 

CRITICAL ERROR: MuMax3 failed to run on this GPU.
Diagnostics (STDERR):
 
No binary for GPU. Your nvidia driver may be out-of-date




In [ ]:
# ## 5. 3D Visualization and Animation
#
# Read .ovf files using ubermag/discretisedfield and animate 3D magnetization.

import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
import discretisedfield as df
from IPython.display import Video, display

# MuMax3 outputs to a directory named after the script
OUTPUT_DIR = 'simulation.out'

def get_m_files():
    # Look for m*.ovf files in the output directory
    if not os.path.exists(OUTPUT_DIR):
        return []
    files = sorted(glob.glob(os.path.join(OUTPUT_DIR, 'm*.ovf')))
    return files

def read_magnetization(filename):
    try:
        # Use from_file (standard in newer ubermag/discretisedfield)
        field = df.Field.from_file(filename)
        return field.array  # Returns (Nx, Ny, Nz, 3) numpy array
    except Exception as e:
        print(f"Failed to read {filename}: {e}")
        return None

# Setup Figure
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

def animate(i):
    files = get_m_files()
    if not files:
        return

    # Loop animation if fewer frames than steps
    idx = i % len(files)
    fname = files[idx]

    m_data = read_magnetization(fname)
    if m_data is None: return

    # Downsample for cleaner visualization (plot every nth arrow)
    ds = 4 # Downsample factor
    mx = m_data[::ds, ::ds, :, 0]
    my = m_data[::ds, ::ds, :, 1]
    mz = m_data[::ds, ::ds, :, 2]

    # Coordinates
    nx, ny, nz = mx.shape
    x, y, z = np.meshgrid(np.arange(nx), np.arange(ny), np.arange(nz), indexing='ij')

    ax.clear()
    # Plot quiver
    q = ax.quiver(x, y, z, mx, my, mz, length=0.8, normalize=True, cmap='coolwarm')

    ax.set_title(f'Magnetization State: {os.path.basename(fname)}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_zlim(-1, max(nz, 2))

# Create Animation
print("Initializing animation...")
files = get_m_files()
if files:
    print(f"Found {len(files)} output files.")
    ani = FuncAnimation(fig, animate, frames=len(files), interval=200, blit=False)

    video_name = 'magnetization_dynamics.mp4'
    try:
        print("Rendering video (this may take a moment)...")
        ani.save(video_name, writer='ffmpeg', fps=10)
        print(f"Animation saved to {video_name}")

        # Close the static plot to prevent it from displaying
        plt.close(fig)

        # Display the video player
        print("Displaying video:")
        display(Video(video_name, embed=True, html_attributes="controls autoplay loop"))

    except Exception as e:
        print(f"Could not save or display video: {e}")
else:
    print("No simulation data found yet. Run Step 4, then run this cell again.")

Initializing animation...
Found 52 output files.
Rendering video (this may take a moment)...
Animation saved to magnetization_dynamics.mp4
Displaying video:
